In [19]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
torch.manual_seed(0)

In [20]:
import csv
import os
import pandas as pd

path = os.path.join("../../data", "integrated_data_20251030_195505.csv")
df = pd.read_csv(path)

print("DataFrame shape:", df.shape)
print("Columns:", list(df.columns))
print("\nFirst 5 rows:")
print(df.head())

# Try to drop timestamp and hand_label
df_numeric = df.drop(columns=['timestamp', 'hand_label'], errors='ignore')

# DEBUG: Check what columns remain after dropping
print("\n=== DEBUG: Columns after dropping ===")
print(list(df_numeric.columns))
print("\nData types:")
print(df_numeric.dtypes)

# Only select truly numeric columns
numeric_columns = ['iteration', 'env0', 'raw0', 'env1', 'raw1', 'env2', 'raw2', 'env3', 'raw3', 
                   'thumb_tip', 'thumb_base', 'index', 'middle', 'ring', 'pinky']

# Let pandas automatically select only numeric columns
data = df_numeric.select_dtypes(include=[np.number]).values.astype(np.float32)
print(f"\nActually numeric columns: {list(df_numeric.select_dtypes(include=[np.number]).columns)}")

print(f"\nNumeric data shape: {data.shape}")
print("First 5 rows of numeric data:")
print(data[:5])

DataFrame shape: (1000, 17)
Columns: ['timestamp', 'iteration', 'env0', 'raw0', 'env1', 'raw1', 'env2', 'raw2', 'env3', 'raw3', 'thumb_tip', 'thumb_base', 'index', 'middle', 'ring', 'pinky', 'hand_label']

First 5 rows:
                    timestamp  iteration   env0     raw0      env1     raw1  \
0  2025-10-30T19:57:11.124404          0  495.0  18.0000  494.0000  63.0000   
1  2025-10-30T19:57:11.248486          1  494.0  18.0000  494.0000  63.0000   
2  2025-10-30T19:57:11.331993          2  494.0  19.0000  495.0000  73.0000   
3  2025-10-30T19:57:11.521735          3  494.0   0.7823    0.5059   0.6457   
4  2025-10-30T19:57:11.635704          4  495.0  18.0000  493.0000  68.0000   

       env2     raw2      env3  raw3 thumb_tip thumb_base   index  middle  \
0  492.0000  59.0000  494.0000    23    0.8696     0.4957  0.6387  0.7544   
1  494.0000  38.0000  494.0000    23    0.8148     0.5007  0.6152  0.7285   
2  495.0000  60.0000  494.0000    23    0.7807     0.4759  0.6399  0.7707 

In [21]:
sensor_columns = ['iteration', 'env0', 'raw0', 'env1', 'raw1', 'env2', 'raw2', 'env3', 'raw3']
finger_columns = ['thumb_tip', 'thumb_base', 'index', 'middle', 'ring', 'pinky']

actual_numeric_cols = list(df_numeric.select_dtypes(include=[np.number]).columns)

# Find indices based on the actual data array
sensor_indices = [actual_numeric_cols.index(col) for col in sensor_columns if col in actual_numeric_cols]
finger_indices = [actual_numeric_cols.index(col) for col in finger_columns if col in actual_numeric_cols]


def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length, :]  
        y = data[i + seq_length, :]   
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Split data into 80% train, 20% test BEFORE creating sequences
split_ratio = 0.8
split_index = int(len(data) * split_ratio)

train_data = data[:split_index]
test_data = data[split_index:]

print(f"Total data points: {len(data)}")
print(f"Train data points: {len(train_data)}")
print(f"Test data points: {len(test_data)}")

# Create sequences separately for train and test
seq_length = 15
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Convert to PyTorch tensors
trainX = torch.tensor(X_train[:, :, sensor_indices], dtype=torch.float32)
trainY = torch.tensor(y_train[:, finger_indices], dtype=torch.float32)
testX = torch.tensor(X_test[:, :, sensor_indices], dtype=torch.float32)
testY = torch.tensor(y_test[:, finger_indices], dtype=torch.float32)

print(f"\ntrainX shape: {trainX.shape}")
print(f"trainY shape: {trainY.shape}")
print(f"testX shape: {testX.shape}")
print(f"testY shape: {testY.shape}")


Total data points: 1000
Train data points: 800
Test data points: 200

trainX shape: torch.Size([785, 15, 8])
trainY shape: torch.Size([785, 0])
testX shape: torch.Size([185, 15, 8])
testY shape: torch.Size([185, 0])


In [22]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(
                0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(
                0), self.hidden_dim).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Take last time step
        return out, hn, cn

In [ ]:
model = LSTMModel(input_dim=9, hidden_dim=200, layer_dim=4, output_dim=6)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [24]:
num_epochs = 100
h0, c0 = None, None
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs, h0, c0 = model(trainX, h0, c0)

    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    h0, c0 = h0.detach(), c0.detach()
    
    model.eval()
    with torch.no_grad():
        val_outputs, _, _ = model(testX, None, None)
        val_loss = criterion(val_outputs, testY)
        val_losses.append(val_loss.item())

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

c:\Users\tobia\Documents\Cogito\Grip\Grip\.venv\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a target size (torch.Size([785, 0])) that is different to the input size (torch.Size([785, 6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (6) must match the size of tensor b (0) at non-singleton dimension 1

In [ ]:
# Plot training and validation losses
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='Train Loss', alpha=0.8)
plt.plot(val_losses, label='Validation Loss', alpha=0.8)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nFinal Train Loss: {train_losses[-1]:.4f}")
print(f"Final Validation Loss: {val_losses[-1]:.4f}")

In [ ]:
# Save the trained model
model_save_path = "../../models/lstm_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
model.eval()
with torch.no_grad():
    predicted, _, _ = model(testX, None, None)


predicted = predicted.detach().numpy() 
original = testY.numpy()  


# Create plots for each finger
plt.figure(figsize=(18, 12))
for i, finger in enumerate(finger_columns):
    plt.subplot(3, 2, i + 1)  # Create a subplot for each finger
    plt.plot(predicted[:, i], label='Predicted', linestyle='--')
    plt.plot(original[:, i], label='Original', alpha=0.7)
    plt.title(f"Finger: {finger}")
    plt.xlabel("Time Step")
    plt.ylabel("Value")
    plt.legend()

plt.tight_layout()
plt.show()